# MAE 549 Final Project


## Kevin Andrade, Nathan, Spilker, Lap Lam, Jose Yanez

## Lab Setup

In [1]:
import time
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
import numpy as np
import cv2

group_number = 80
camera = 1

ModuleNotFoundError: No module named 'cflib'

## Controller Helper Functions

In [4]:
def position_estimate(scf):
    """ Calculates position of the drone """
    log_config = LogConfig(name='Position Estimate', period_in_ms=500)
    log_config.add_variable('stateEstimate.x', 'float')
    log_config.add_variable('stateEstimate.y', 'float')
    log_config.add_variable('stateEstimate.z', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['stateEstimate.x']
            y = data['stateEstimate.y']
            z = data['stateEstimate.z']
            break
    print(x, y, z)
    return x, y, z


###############################################################################

def ascend_and_hover(cf):
    """ Ascend and hover """
    print('Ascending to Hover:')
    for y in range(5):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    return

###############################################################################

def ascend(cf):
    """ Ascend """
    print('Ascending:')
    for y in range(15):
        cf.commander.send_hover_setpoint(0, 0, 0, (25 + y) / 50)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.8)
        time.sleep(0.1)
    return

###############################################################################

def hover_and_descend(cf):
    """ Hover and descend """
    print('Descending:')
    # Hover at 0.5 meters:
    for _ in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.8)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (8 - y) / 10)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

    
###############################################################################

def hover(cf):
    """ Hover """
    print('Hovering:')
    for _ in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.8)
        time.sleep(0.1)
    return


###############################################################################

def set_PID_controller(cf):
    """ Set PID controller for drone """
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


###############################################################################


def controller(cf, box_x, box_y, box_width, box_height, x_curr, y_curr):
    """
    cf: crazyflie instance
    box_x: x coordinate of the center of the bounding box in the image
    box_y: y coordinate of the center of the bounding box in the image
    box_width: width of the bounding box in the image
    box_height: height of the bounding box in the image
    x_curr: current x position
    y_curr: current y position
    
    Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
    Return False to indicate continuing to follow the target, new x, new y.
    """
    
    # Exit condition/method using size of the bounding box
    size = box_width*box_height
    exitloop = False
    if abs(box_x) < 0.2:
        exitloop = True
    
    # Determine the x and y to use in send_position_setpoint() 
    y_factor = 0.05 
    y_command = y_curr - (box_x * y_factor)
    
    # Set position
    cf.commander.send_position_setpoint(x_curr, y_command, 0.8, 0)
    new_x = x_command
    new_y = y_command
    
    return exitloop, new_x, new_y  


###############################################################################


def go_left(cf, x_curr, y_drone, offset, y_max):
    """ Move to far left boundary """
    print('Moving Left!')
    margin = 0.35
    y_curr = y_drone + offset
    step = (y_max - margin - y_curr)/30
    for y in range(30):
        y_command = y_curr + (step * y) - offset
        cf.commander.send_position_setpoint(x_curr, y_command, 0.8, 0)
        time.sleep(0.1)
        y_drone = y_command
    return y_drone



###############################################################################

def step_right(cf, x_curr, y_drone, y_min, y_max):
    """ Scan right for person """
    print('Searching for Person!')    
    margin = 0.35
    step = (y_min - y_max - margin) / 50
    cf.commander.send_position_setpoint(x_curr, y_drone + step, 0.8, 0)
    y_drone = y_drone + step
    time.sleep(0.1)
    return y_drone



## Image Analysis Helper Functions

In [3]:
def findGreatesContour(contours):
    """ Calculates greatest contour value in image"""
    
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    return largest_area, largest_contour_index

###############################################################################


def check_contours(frame):
    """ Analyses contour values of image"""
    
    # Adjust frame values
    frame = frame[100:380, 0:640]
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red BGR value ranges 
    lb = np.array([0,150,150])
    ub = np.array([70,250,250])

    # Adjust to HSV values
    mask = cv2.inRange(hsv,lb,ub)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    # Check contour values 
    largest_area, largest_contour_index = findGreatesContour(contours)
    if largest_area > 4000:
        return (True, find_midpoint(contours[largest_contour_index], frame))
    else:
        return (False, -1)
    
    
###############################################################################


def find_midpoint(contour, frame):
    """ Finds the midpoint of the contour """    
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2)
    return (x + (w / 2))


###############################################################################


def detection_center(detection):
    """ Computes the center x, y coordinates of the object"""
    center_x = (detection[3] + detection[5]) / 2.0 - 0.5
    center_y = (detection[4] + detection[6]) / 2.0 - 0.5
    return (center_x, center_y)


###############################################################################

def norm(vec):
    """ Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)


###############################################################################

def closest_detection(detections):
    """ Find the detection closest to the image center"""
    if len(detections) == 0:
        return None
    min_dist = np.Inf
    min_detection = None
    for detection in detections:
        center = detection_center(detection)

        dist = norm(center)
        if dist < mindist:
            min_dist = dist
            min_detection = detection
            
    return min_detection


## Main Method for Obstacle Avoidance

In [2]:
# Extract DNN resources 
with open('Lab9_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')
    
# Initialize colors for each class
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# Load the DNN model for object recognition
model = cv2.dnn.readNet(model='Lab9_Supplement/frozen_inference_graph.pb',
                        config='Lab9_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:

        
################################### Drone Initialization #########################

        # Initialize drone position values 
        cf = scf.cf
        y_curr = 0.0
        x_curr = 0.0
        y_drone = 0.0
        offset = y_curr - y_drone
        
        # Initialize static environment values
        y_min = -0.787
        y_max = 0.787
        drone_radius = 0.04
        table_dist = 3.05
        
        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0
        cap = cv2.VideoCapture(camera)
        


########################### Obstacle Avoidance Algorithm #########################
        
        # Check for goal 
        while(x_curr < table_dist):
            ret, frame = cap.read()
            
            
            # Check total elapsed time
            elapsed = time.time() - t
            if(elapsed > 0.005):

                # Initial drone ascend 
                if(ascended_bool==0):
                    set_PID_controller(cf)
                    ascend_and_hover(cf)
                    ascended_bool = 1
                # Run object detection algorithm 
                else:
                    # Check for matching contours near drone
                    obj_near = check_contours(frame)
                    if (obj_near[0]):
                        loc_mid = obj_near[1] - 320
                        y_step = 0.005
                        at_boundary = False
                        
                        # Check for right boundary 
                        if y_curr < y_min:
                            y_command = y_drone + y_step
                            at_boundary = True
                            
                        # Check for left boundary 
                        if y_curr > y_max:
                            y_command = y_drone-y_step
                            at_boundary = True
                           
                        # Within bounds
                        if not at_boundary:
                            if loc_mid < 0:
                                y_command = y_drone-y_step
                            else:
                                y_command = y_drone+y_step
                        
                        # Update drone positions 
                        cf.commander.send_position_setpoint(x_curr, y_command, 0.5, 0)
                        y_drone = y_command
                        y_curr = y_command + offset
                    
                    # When contours are not detected
                    else:
                        step = 0.01
                        x_command = x_curr + step
                        cf.commander.send_position_setpoint(x_command, y_drone, 0.5, 0)
                        x_curr = x_command
                    
                t = time.time()

        # Ascend and begin book tracking:
        ascend(cf)
        y_drone = go_left(cf, x_curr, y_drone, offset, y_max)
        
        confidence = 0.3
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False
             
        
        
############################### Object Detection (Person) #########################

        # Update drone position
        y_curr = y_drone
        # Detect objects in each frame of the video
        while cap.isOpened() and not exit_loop:
            
            # Try to read image
            ret, frame = cap.read()
            if ret:
                tracking_label = 1
                image = frame
                image_height, image_width, _ = image.shape

                # Create blob from image
                blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
                                             swapRB=True)

                # Forward propagate image
                model.setInput(blob)
                detections = model.forward()

                # Select detections that match selected class label
                matching_detections = [d for d in detections[0, 0] if d[1] == tracking_label]

                # Select confident detections
                confident_detections = [d for d in matching_detections if d[2] > confidence]

                # Get detection closest to center of field of view and draw it
                det = closest_detection(confident_detections) # This relies on the function you wrote above
                
                if det is not None:
                    # Get the class id
                    class_id = det[1]
                    # Map the class id to the class 
                    class_name = class_names[int(class_id)-1]
                    color = COLORS[int(class_id)]
                    # Get the bounding box coordinates
                    box_x = det[3] * image_width
                    box_y = det[4] * image_height
                    # Get the bounding box width and height
                    box_width = det[5] * image_width
                    box_height = det[6] * image_height
                    # Draw a rectangle around each detected object
                    cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width),
                                                                    int(box_height)),
                                                                    color, 
                                                                    thickness=2)
                    # Put the class name text on the detected object
                    cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), 
                                                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                                                    1, color, 2)

                # If nothing is detected, hover
                if det is None:
                    print('No detection...stepping right')
                    y_curr = step_right(cf, x_curr, y_curr, y_min, y_max)

                # Otherwise  move towards target
                else:
                    print('Detection...tracking')
                    _, _, _, box_x, box_y, box_width, box_height = det
                    box_x, box_y = detection_center(det)
                    exit_loop, x_curr, y_curr = controller(cf, box_x, box_y, box_width, box_height, x_curr, y_curr)
                    
            else:
                print('No Image!!')     
        cap.release()
        
                
        # Descend and stop all motion:
        hover_and_descend(cf)
        cv2.destroyAllWindows()
        

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/16/2M
radio://0/16/2M
radio://0/80/2M
Initializing PID Controller
Going Left!
-0.07
-0.0531
-0.036199999999999996
-0.01929999999999999
-0.0023999999999999855
0.014500000000000013
0.031400000000000025
0.04830000000000004
0.06520000000000004
0.08210000000000003
0.09900000000000003
0.11590000000000006
0.13280000000000006
0.14970000000000006
0.16660000000000008
0.18350000000000005
0.20040000000000008
0.2173000000000001
0.23420000000000007
0.2511000000000001
0.26800000000000007
0.2849000000000001
0.3018000000000001
0.3187000000000001
0.3356000000000001
0.35250000000000015
0.3694000000000001
0.38630000000000014
0.40320000000000017
0.42010000000000014
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detection...stepping right
no detectio